## Overview of the Data Set

In this project, we'll continue working with the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of facts about countries. The Factbook contains demographic information for each country in the world, including:

- `populatio`n - The population as of 2015
- `population_growth` - The annual population growth rate, as a percentage
- `area` - The total land and water area

You can download the Factbook as a SQLite database [from GitHub](https://github.com/factbook/factbook.sql) if you want to work with it on your own computer. In this guided project, we'll be working with Python and the SQLite command line tool (SQLite Command Shell) to connect to the database, extract data, and perform analysis on the data.

## SQLite Command Shell

SQLite is a relational database management system that enables us to create databases and query them using SQL syntax. SQLite is simpler than full database systems like MySQL and PostgreSQL. It's good for cases where ease of use is more important than performance. Each SQLite database is stored as a single file, making it easy to transport.

The Factbook database is in the file `factbook.db`. The `db` at the end is a file extension that's short for database.

We can open the Factbook database in the SQLite Command Shell by navigating to the same folder as factbook.db, then typing `sqlite3 factbook.db` on the command line. This enables us to manage the database and run SQL queries.

## Using Python with SQLite

The [`sqlite3` library](https://docs.python.org/3/library/sqlite3.html), which comes with Python by default, allows us to connect to SQLite databases. To do this, we open a database connection, then create an object that can run queries.

For example, this code will let us connect to `factbook.db` and select all of the rows:

```python
import sqlite3
conn = sqlite3.connect(r'C:\Users\USER\Downloads\The Ultimate Plan\Jupyter Notebook\Datasets\factbook.db')

c = conn.cursor()
c.execute('SELECT * FROM facts;')

print(c.fetchall())
```

The code above creates a [`Connection`](https://docs.python.org/3/library/sqlite3.html#connection-objects) object. We then create a [`Cursor`](https://docs.python.org/3/library/sqlite3.html#cursor-objects) instance, which can execute queries. Finally, we execute a query and display the results using the `print` function. To learn more about the `sqlite3` library, read the package documentation [on the official Python website](https://docs.python.org/3/library/sqlite3.html).

In [1]:
import sqlite3

conn = sqlite3.connect(r'C:\Users\USER\Downloads\The Ultimate Plan\Jupyter Notebook\Datasets\factbook.db')

cur = conn.cursor()
query = "select name, population from facts where population != 'None' order by population asc limit 10;"
cur.execute(query)

print(cur.fetchall())

[('Antarctica', 0), ('Pitcairn Islands', 48), ('Cocos (Keeling) Islands', 596), ('Holy See (Vatican City)', 842), ('Niue', 1190), ('Tokelau', 1337), ('Christmas Island', 1530), ('Svalbard', 1872), ('Norfolk Island', 2210), ('Falkland Islands (Islas Malvinas)', 3361)]


## Computing Population Projections

You can read the results of a SQL query into a pandas dataframe using the `read_sql_query` function, which the [official pandas website documents](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_query.html). The `read_sql_query` function takes a SQL query string and a connection object, and returns a dataframe containing all of the rows and columns from the query.

When pandas reads in columns this way, it automatically uses the column types from the original the database. Blank entries in the database (like the ones in the `area_land` column) will have `NaN` values in a dataframe, which means "Not a Number." This is because pandas can't have blanks in numeric columns; it uses `NaN` to signify invalid or missing values instead.

You can learn more about how to work with missing data in the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/missing_data.html). For now, we'll just use the `dropna` method with the `axis=0` argument, which will drop any rows that have `NaN` values.

In [11]:
import pandas as pd
import math

facts = pd.read_sql_query('select * from facts', conn)
facts = facts.dropna(axis=0)

def predict_2015(row):
    n = row['population']*(math.e**((row['population_growth']/100)*35))
    return round(n,0)

facts['2050_pop'] = facts.apply(predict_2015, axis=1)
facts

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,created_at,updated_at,2050_pop
0,1,af,Afghanistan,652230.0,652230.0,0.0,32564342.0,2.32,38.57,13.89,1.51,2015-11-01 13:19:49.461734,2015-11-01 13:19:49.461734,73348194.0
1,2,al,Albania,28748.0,27398.0,1350.0,3029278.0,0.30,12.92,6.58,3.30,2015-11-01 13:19:54.431082,2015-11-01 13:19:54.431082,3364651.0
2,3,ag,Algeria,2381741.0,2381741.0,0.0,39542166.0,1.84,23.67,4.31,0.92,2015-11-01 13:19:59.961286,2015-11-01 13:19:59.961286,75291526.0
3,4,an,Andorra,468.0,468.0,0.0,85580.0,0.12,8.13,6.96,0.00,2015-11-01 13:20:03.659945,2015-11-01 13:20:03.659945,89251.0
4,5,ao,Angola,1246700.0,1246700.0,0.0,19625353.0,2.78,38.78,11.49,0.46,2015-11-01 13:20:08.625072,2015-11-01 13:20:08.625072,51926136.0
5,6,ac,Antigua and Barbuda,442.0,442.0,0.0,92436.0,1.24,15.85,5.69,2.21,2015-11-01 13:20:13.049627,2015-11-01 13:20:13.049627,142667.0
6,7,ar,Argentina,2780400.0,2736690.0,43710.0,43431886.0,0.93,16.64,7.33,0.00,2015-11-01 13:20:18.233063,2015-11-01 13:20:18.233063,60141124.0
7,8,am,Armenia,29743.0,28203.0,1540.0,3056382.0,0.15,13.61,9.34,5.80,2015-11-01 13:20:23.048753,2015-11-01 13:20:23.048753,3221129.0
8,9,as,Australia,7741220.0,7682300.0,58920.0,22751014.0,1.07,12.15,7.14,5.65,2015-11-01 13:20:28.186341,2015-11-01 13:20:28.186341,33085983.0
9,10,au,Austria,83871.0,82445.0,1426.0,8665550.0,0.55,9.41,9.42,5.56,2015-11-01 13:20:33.093597,2015-11-01 13:20:33.093597,10505043.0


## Summing Columns to Compute Total Area

We can add up all of the values in a column by using the `SUM` function in a SQL query. For example, we can calculate the total `population` with this query:

```sql
SELECT SUM(population) from facts;
```

We can also add a `WHERE` clause, like this:

```sql
SELECT SUM(population) from facts WHERE area_land != "";
```

In [2]:
query = "select sum(area_land) / sum(area_water) from facts where (area_land != '') and (area_water != '');"
total = cur.execute(query).fetchone()
print(total[0]) 

27


Here are a few more interesting questions to answer to:

- Which countries will lose population over the next 35 years?
- Which countries have the lowest and highest population densities?
- Which countries receive the most immigrants? Which countries lose the most imigrants?